### _VectorRetriever_ ###

In [1]:
# Creating a Sample Document 
from langchain_core.documents import Document

In [2]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]= os.getenv("HF_TOKEN")

In [5]:
model = ChatGroq(groq_api_key= groq_api_key,model = 'llama3-8b-8192')
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fd191eda350>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fd191edb7c0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(
    documents,embedding = embeddings
)
vectorstore

In [ ]:
vectorstore.similarity_search("cat")


[Document(id='fcf145bb-c8b3-41ca-b6df-8779a56ee785', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='72a346e2-0b41-429f-a6ce-59aba732f50e', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='19f746b9-d05c-4c35-b00e-49c09e1fd809', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='f3f53649-7fa2-4d91-9209-b5d28f9041de', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [13]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='fcf145bb-c8b3-41ca-b6df-8779a56ee785', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351056814193726),
 (Document(id='72a346e2-0b41-429f-a6ce-59aba732f50e', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351056814193726),
 (Document(id='19f746b9-d05c-4c35-b00e-49c09e1fd809', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740900039672852),
 (Document(id='f3f53649-7fa2-4d91-9209-b5d28f9041de', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740900039672852)]

In [16]:
from typing import List
from langchain_core.runnables import RunnableLambda
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='fcf145bb-c8b3-41ca-b6df-8779a56ee785', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='19f746b9-d05c-4c35-b00e-49c09e1fd809', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [19]:
retriever = vectorstore.as_retriever(search_type = 'similarity',
search_kwargs = {'k':1})
retriever.batch(['cat','dog'])

[[Document(id='fcf145bb-c8b3-41ca-b6df-8779a56ee785', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='19f746b9-d05c-4c35-b00e-49c09e1fd809', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [23]:
##RAG

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message="""
Answer the question using the provided context only
{question}
context:{context}
"""
prompt = ChatPromptTemplate.from_messages(
    [("human",message)]
)
rag_chain = {'context':retriever,'question':RunnablePassthrough()}|prompt|model

response = rag_chain.invoke('Tell me about dogs?')
print(response.content)

According to the context, dogs are great companions, known for their loyalty and friendliness.
